

```
# This is formatted as code
```

Injury Model

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
playlist_df = pd.read_csv('/content/drive/MyDrive/Injury_Prediction_Analysis/Project-4-Dataset/PlayList.csv')
playlist_df.head()

,PlayerKey,GameID,PlayKey,RosterPosition,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Position,PositionGroup
0,26624,26624-1,26624-1-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,1,QB,QB
1,26624,26624-1,26624-1-2,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,2,QB,QB
2,26624,26624-1,26624-1-3,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Rush,3,QB,QB
3,26624,26624-1,26624-1-4,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Rush,4,QB,QB
4,26624,26624-1,26624-1-5,Quarterback,1,1,Outdoor,Synthetic,63,Clear and warm,Pass,5,QB,QB


In [14]:
playlist_df["PlayerGamePlay"].value_counts()

,count
PlayerGamePlay,
1,5712
2,5698
3,5679
4,5650
5,5623
...,...
98,10
99,6
100,4


In [6]:
playlist_df['StadiumType'].unique()

array(['Outdoor', 'Indoors', 'Oudoor', 'Outdoors', 'Open', 'Closed Dome',
       'Domed, closed', nan, 'Dome', 'Indoor', 'Domed',
       'Retr. Roof-Closed', 'Outdoor Retr Roof-Open', 'Retractable Roof',
       'Ourdoor', 'Indoor, Roof Closed', 'Retr. Roof - Closed', 'Bowl',
       'Outddors', 'Retr. Roof-Open', 'Dome, closed', 'Indoor, Open Roof',
       'Domed, Open', 'Domed, open', 'Heinz Field', 'Cloudy',
       'Retr. Roof - Open', 'Retr. Roof Closed', 'Outdor', 'Outside'],
      dtype=object)

In [7]:
playlist_df['StadiumType'] = playlist_df['StadiumType'].str.replace(r'Oudoor|Outdoors|Ourdoor|Outddors|Outdor|Outside|Bowl|Heinz Field|Open|Domed, open|Outdoor Retr Roof-Open|Retr. Roof-Open|Retr. Roof - Open|Outdoor Retr Roof-Outdoor|Retr. Roof-Outdoor|Cloudy', 'Outdoor', regex=True)
playlist_df['StadiumType'] = playlist_df['StadiumType'].str.replace(r'Indoors|Indoor, Roof Closed|Indoor, Open Roof| Closed Dome|Domed, closed|Domed, closed|Domed|Dome|Retr. Roof-Closed|Retr. Roof - Closed|Retr. Roof Closed|Indoor, Outdoor Roof|Closed Indoor|Retractable Roof|Indoor, closed|Indoor, Outdoor', 'Indoor', regex=True)

In [8]:
playlist_df['StadiumType'].unique()

array(['Outdoor', 'Indoor', 'Closed Indoor', nan, 'Indoor, closed',
       'Indoor, Outdoor'], dtype=object)

In [9]:
playlist_df['StadiumType'] = playlist_df['StadiumType'].str.replace(r'Closed Indoor|Indoor, closed|Indoor, Outdoor', 'Indoor', regex=True)

In [10]:
playlist_df['StadiumType'].unique()

array(['Outdoor', 'Indoor', nan], dtype=object)

In [11]:
playlist_df['StadiumType'].value_counts()

,count
StadiumType,
Outdoor,189480
Indoor,60615


In [12]:
playlist_df['Weather'].unique()

array(['Clear and warm', 'Mostly Cloudy', 'Sunny', 'Clear', 'Cloudy',
       'Cloudy, fog started developing in 2nd quarter', 'Rain',
       'Partly Cloudy', 'Mostly cloudy', 'Cloudy and cold',
       'Cloudy and Cool', 'Rain Chance 40%', 'Controlled Climate',
       'Sunny and warm', 'Partly cloudy', 'Clear and Cool',
       'Clear and cold', 'Sunny and cold', 'Indoor', nan, 'Partly Sunny',
       'N/A (Indoors)', 'Mostly Sunny', 'Indoors', 'Clear Skies',
       'Partly sunny', 'Showers', 'N/A Indoor', 'Sunny and clear', 'Snow',
       'Scattered Showers', 'Party Cloudy', 'Clear skies',
       'Rain likely, temps in low 40s.', 'Hazy', 'Partly Clouidy',
       'Sunny Skies', 'Overcast', 'Cloudy, 50% change of rain', 'Fair',
       'Light Rain', 'Partly clear', 'Mostly Coudy', '10% Chance of Rain',
       'Cloudy, chance of rain', 'Heat Index 95',
       'Sunny, highs to upper 80s', 'Sun & clouds',
       'Heavy lake effect snow', 'Mostly sunny', 'Cloudy, Rain',
       'Sunny, Windy', '

In [16]:
cloudy = ['Hazy', 'Cloudy.', 'Overcast', 'Mostly Cloudy',
          'Cloudy, fog started developing in 2nd quarter', 'Partly Cloudy',
          'Mostly cloudy', 'Rain Chance 40%',' Partly cloudy', 'Party Cloudy',
          'Rain likely, temps in low 40s', 'Partly Clouidy', 'Mostly Coudy', '10% Chance of Rain',
          'Cloudy, chance of rain', '30% Chance of Rain', 'cloudy', 'Coudy',
         'Cloudy fog started developing in 2nd quarter',
         'Cloudy 50% change of rain', 'Cloudy and cold',
         'Cloudy and Cool', 'Partly cloudy','Cloudy.']

clear = ['Clear, Windy',' Clear to Cloudy', 'Clear, highs to upper 80s',
             'Clear and clear','Partly sunny', 'Heat Index 95', 'Cold',
             'Clear, Windy', 'Clear skies', 'Sunny', 'Partly Sunny', 'Mostly Sunny', 'Clear Skies',
             'Sunny Skies', 'Partly clear', 'Fair', 'Sunny, highs to upper 80s', 'Sun & clouds', 'Mostly sunny','Sunny, Windy',
             'Mostly Sunny Skies', 'Clear and Sunny', 'Clear and sunny','Clear to Partly Cloudy', 'Clear Skies',
            'Clear and cold', 'Clear and warm', 'Clear and Cool', 'Sunny and cold', 'Sunny and warm', 'Sunny and clear']

rainy = ['Rainy', 'Scattered Showers', 'Showers', 'Cloudy Rain', 'Light Rain', 'Rain shower',
             'Rain likely, temps in low 40s.', 'Cloudy, Rain',
             'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.', 'Cloudy 50% change of rain',
             'Cloudy, 50% change of rain']

snow = ['Heavy lake effect snow', 'Cloudy, light snow accumulating 1-3"']

indoor = ['Controlled Climate', 'Indoors', 'N/A Indoor', 'N/A (Indoors)']

weather_buckets = {
    'Cloudy': cloudy,
    'Clear': clear,
    'Rainy': rainy,
    'Snowy': snow,
    'Indoor': indoor
}

def categorize_weather(weather):
  for bucket, conditions in weather_buckets.items():
      if weather in conditions:
          return bucket

playlist_df['Weather'] = playlist_df['Weather'].apply(categorize_weather)

playlist_df['Weather'].unique()

array(['Clear', 'Cloudy', None, 'Indoor', 'Rainy', 'Snowy'], dtype=object)

In [17]:
injury_df = pd.read_csv('/content/drive/MyDrive/Injury_Prediction_Analysis/Project-4-Dataset/InjuryRecord.csv')
injury_df.head()

,PlayerKey,GameID,PlayKey,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42
0,39873,39873-4,39873-4-32,Knee,Synthetic,1,1,1,1
1,46074,46074-7,46074-7-26,Knee,Natural,1,1,0,0
2,36557,36557-1,36557-1-70,Ankle,Synthetic,1,1,1,1
3,46646,46646-3,46646-3-30,Ankle,Natural,1,0,0,0
4,43532,43532-5,43532-5-69,Ankle,Synthetic,1,1,1,1


In [18]:
playlist_injury_df = injury_df.set_index('GameID').join(playlist_df.set_index('GameID'), how = 'outer',lsuffix='_left', rsuffix='_right')
playlist_injury_df.head()

,PlayerKey_left,PlayKey_left,BodyPart,Surface,DM_M1,DM_M7,DM_M28,DM_M42,PlayerKey_right,PlayKey_right,...,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Position,PositionGroup
GameID,,,,,,,,,,,,,,,,,,,,,
26624-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26624,26624-1-1,...,1,1,Outdoor,Synthetic,63,Clear,Pass,1,QB,QB
26624-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26624,26624-1-2,...,1,1,Outdoor,Synthetic,63,Clear,Pass,2,QB,QB
26624-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26624,26624-1-3,...,1,1,Outdoor,Synthetic,63,Clear,Rush,3,QB,QB
26624-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26624,26624-1-4,...,1,1,Outdoor,Synthetic,63,Clear,Rush,4,QB,QB
26624-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26624,26624-1-5,...,1,1,Outdoor,Synthetic,63,Clear,Pass,5,QB,QB


In [19]:
playlist_injury_df['DM_M1'] = playlist_injury_df['DM_M1'].fillna(0).astype(int)
playlist_injury_df['DM_M7'] = playlist_injury_df['DM_M7'].fillna(0).astype(int)
playlist_injury_df['DM_M28'] = playlist_injury_df['DM_M28'].fillna(0).astype(int)
playlist_injury_df['DM_M42'] = playlist_injury_df['DM_M42'].fillna(0).astype(int)
playlist_injury_df['Injury Severity'] = playlist_injury_df['DM_M1'] + playlist_injury_df['DM_M7'] + playlist_injury_df['DM_M28'] + playlist_injury_df['DM_M42']
playlist_injury_df['Injury'] = np.where(playlist_injury_df['Injury Severity'] > 0, 1, 0)
playlist_injury_df.drop(columns=['PlayerKey_left', 'PlayKey_right','PlayKey_left', 'PlayerKey_right', 'DM_M1', 'DM_M7', 'DM_M28', 'DM_M42', "Surface", "BodyPart"],axis=1,inplace=True)
playlist_injury_df.head()

,RosterPosition,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Position,PositionGroup,Injury Severity,Injury
GameID,,,,,,,,,,,,,
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Pass,1,QB,QB,0,0
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Pass,2,QB,QB,0,0
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Rush,3,QB,QB,0,0
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Rush,4,QB,QB,0,0
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Pass,5,QB,QB,0,0


In [25]:
playlist_injury_df.drop(columns=["Position", "PositionGroup"],axis=1,inplace=True)
playlist_injury_df.head()

,RosterPosition,PlayerDay,PlayerGame,StadiumType,FieldType,Temperature,Weather,PlayType,PlayerGamePlay,Injury Severity,Injury
GameID,,,,,,,,,,,
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Pass,1,0,0
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Pass,2,0,0
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Rush,3,0,0
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Rush,4,0,0
26624-1,Quarterback,1,1,Outdoor,Synthetic,63,Clear,Pass,5,0,0


In [32]:
playlist_injury_df["Injury"].value_counts()

,count
Injury,
0,263620
1,3403


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [56]:
y = playlist_injury_df['Injury']
X = playlist_injury_df.drop(columns=['Injury', 'Injury Severity'])

In [57]:
X = pd.get_dummies(X)
X.head()

,PlayerDay,PlayerGame,Temperature,PlayerGamePlay,RosterPosition_Cornerback,RosterPosition_Defensive Lineman,RosterPosition_Kicker,RosterPosition_Linebacker,RosterPosition_Offensive Lineman,RosterPosition_Quarterback,...,PlayType_Extra Point,PlayType_Field Goal,PlayType_Kickoff,PlayType_Kickoff Not Returned,PlayType_Kickoff Returned,PlayType_Pass,PlayType_Punt,PlayType_Punt Not Returned,PlayType_Punt Returned,PlayType_Rush
GameID,,,,,,,,,,,,,,,,,,,,,
26624-1,1,1,63,1,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
26624-1,1,1,63,2,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
26624-1,1,1,63,3,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
26624-1,1,1,63,4,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
26624-1,1,1,63,5,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=38)

In [ ]:
## RANDOM FOREST MODEL

In [59]:
model = RandomForestClassifier(n_estimators=100, random_state=38)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=38)

In [60]:
y_pred = model.predict(X_test)

In [64]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_pred)

In [65]:
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,52584,141
Actual 1,158,522


Accuracy Score : 0.9944012732890178
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52725
           1       0.79      0.77      0.78       680

    accuracy                           0.99     53405
   macro avg       0.89      0.88      0.89     53405
weighted avg       0.99      0.99      0.99     53405



In [63]:
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
print("Feature Importance:\n", feature_importances.sort_values(ascending=False))

Feature Importance:
 PlayerDay                           0.220641
Temperature                         0.177955
PlayerGame                          0.164713
PlayerGamePlay                      0.150926
RosterPosition_Wide Receiver        0.024801
RosterPosition_Safety               0.024336
RosterPosition_Linebacker           0.023319
Weather_Clear                       0.021416
RosterPosition_Cornerback           0.021211
Weather_Cloudy                      0.019796
RosterPosition_Running Back         0.014059
RosterPosition_Offensive Lineman    0.014005
RosterPosition_Defensive Lineman    0.011676
PlayType_Pass                       0.011003
StadiumType_Outdoor                 0.010875
RosterPosition_Tight End            0.009893
PlayType_Rush                       0.009393
Weather_Rainy                       0.009204
Weather_Indoor                      0.007897
StadiumType_Indoor                  0.007798
FieldType_Natural                   0.007457
PlayType_Kickoff                  

In [ ]:
## K NEIGHBORS MODEL

In [45]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [46]:
knn = KNeighborsClassifier(n_neighbors=7)

In [47]:
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=7)

In [48]:
y_pred = knn.predict(X_test_scaled)

In [49]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     52725
           1       0.60      0.24      0.34       680

    accuracy                           0.99     53405
   macro avg       0.80      0.62      0.67     53405
weighted avg       0.99      0.99      0.99     53405



In [21]:
## LOGISTIC REGRESSION MODEL

In [50]:
y = playlist_injury_df['Injury']
X = playlist_injury_df.drop(columns=['Injury', 'Injury Severity'])

In [51]:
X = pd.get_dummies(X)
X.head()

,PlayerDay,PlayerGame,Temperature,PlayerGamePlay,RosterPosition_Cornerback,RosterPosition_Defensive Lineman,RosterPosition_Kicker,RosterPosition_Linebacker,RosterPosition_Offensive Lineman,RosterPosition_Quarterback,...,PlayType_Extra Point,PlayType_Field Goal,PlayType_Kickoff,PlayType_Kickoff Not Returned,PlayType_Kickoff Returned,PlayType_Pass,PlayType_Punt,PlayType_Punt Not Returned,PlayType_Punt Returned,PlayType_Rush
GameID,,,,,,,,,,,,,,,,,,,,,
26624-1,1,1,63,1,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
26624-1,1,1,63,2,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
26624-1,1,1,63,3,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
26624-1,1,1,63,4,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
26624-1,1,1,63,5,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False


In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [54]:
y_pred = model.predict(X_test)

In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     52725
           1       0.00      0.00      0.00       680

    accuracy                           0.99     53405
   macro avg       0.49      0.50      0.50     53405
weighted avg       0.97      0.99      0.98     53405



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
